In [1]:
import numpy as np
from numpy import linalg
import pandas as pd
import math
from sklearn.metrics import roc_auc_score
from copy import copy
from scipy.spatial.distance import euclidean
import warnings
warnings.filterwarnings('ignore')

## 1. Загрузите данные из файла data-logistic.csv. 

Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [2]:
data = pd.read_csv('C:\\Users\\kvanc\\coursera\\files\\data-logistic.csv', header = None)

In [3]:
data.head()

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750


In [4]:
data_classes = data[0]
data_features = data.iloc[:, 1:]

## 2. Убедитесь, что выше выписаны правильные формулы для градиентного спуска. 

Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

In [5]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def distance_vec(vector1, vector2):
    vector1 = np.array(vector1, dtype = 'f')
    vector2 = np.array(vector2, dtype = 'f')
    return linalg.norm(vector1 - vector2)

In [6]:
def f_w1(w1, w2, X, y):
    l = len(y)
    S = 0
    for i in range(0, l):
        S += y[i] * X[1][i] * (1. - sigmoid(y[i] * (w1 * X[1][i] + w2 * X[2][i])))
    return S

def f_w2(w1, w2, X, y):
    l = len(y)
    S = 0
    for i in range(0, l):
        S += y[i] * X[2][i] * (1. - sigmoid(y[i] * (w1 * X[1][i] + w2 * X[2][i])))
    return S

## 3. Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии.

Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

## 4. Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). 

Рекомендуется ограничить сверху число итераций десятью тысячами. 

In [8]:
def gradient_descent(X, y, C, step = 0.1, eps = 1e-5, max_iter = 10000):
    #X = np.matrix(X)
    #y = np.matrix(y)
    l = len(X)
    w1, w2 = 0., 0.
    #w = np.array([w1, w2])
    diff = 1.
    num_iter = 1
    while (diff > eps):
        #hypothesis = y * (X * np.matrix([w1, w2]).T)
        w1_update = w1 + (step / l) * f_w1(w1, w2, X, y) - step * C * w1 
        w2_update = w2 + (step / l) * f_w2(w1, w2, X, y) - step * C * w2 
        diff = distance_vec([w1, w2], [w1_update, w2_update])
        #diff = math.sqrt((w1_update - w1) ** 2 - (w2_update - w2) ** 2)
        w1 = w1_update
        w2 = w2_update
        w = np.matrix([w1, w2])
        if num_iter >= max_iter:
            break
        num_iter += 1
    return [w1_update, w2_update]

## 5. Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? 

Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)). 

In [71]:
descent_without_L2 = gradient_descent(data_features, data_classes, C = 0)
descent_without_L2 

[0.2878116204717764, 0.09198330215925439]

In [72]:
descent_with_L2 = gradient_descent(data_features, data_classes, C = 10)
descent_with_L2

[0.028558754546234223, 0.02478013724973556]

In [73]:
def a(X, w1, w2):
    return 1 / (1 + math.exp(- w1 * X[1] - w2 * X[2]))

In [74]:
y_score_without_L2 = data_features.apply(lambda x: a(x, descent_without_L2[0], descent_without_L2[1]), axis = 1)
y_score_with_L2 = data_features.apply(lambda x: a(x, descent_with_L2[0], descent_with_L2[1]), axis = 1)

In [75]:
auc_without = roc_auc_score(data_classes, y_score_without_L2)
auc_with = roc_auc_score(data_classes, y_score_with_L2)

In [76]:
print(round(auc_without, 3), round(auc_with, 3))

0.927 0.936
